In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import time
from sklearn.ensemble import BaggingClassifier

In [2]:
data_all=pd.read_csv('final_selected_variables.csv',index_col=0)

In [3]:
data_all.head()

,Merch_description_part_count0__max,Merchnum_count0__max,Merchnum_count1__max,Merchnum_count1__sum,Merchnum_count3__max,cardnum-merch_description_part_count14__max,cardnum-merch_description_part_count14__sum,cardnum-merch_description_part_count30__max,cardnum-merch_description_part_count3__sum,cardnum-merch_state_count14__max,...,cardnum-merch_state_merch_zip_count3__sum,cardnum-merch_zip_count14__max,cardnum-merchnum_count14__max,cardnum-merchnum_count14__sum,cardnum-merchnum_count1__max,cardnum-merchnum_count30__max,cardnum-merchnum_count3__max,cardnum-merchnum_count3__sum,Recnum,Fraud
0,-0.510270,-0.497168,-0.507695,-0.252697,-0.542314,-0.461148,-0.187507,-0.482323,-0.155707,-0.504417,...,-0.156323,-0.463631,-0.456162,-0.183189,-0.424413,-0.476666,-0.431276,-0.153873,1,0
1,-0.482932,-0.469230,-0.483051,-0.246306,-0.519227,-0.434575,-0.180833,-0.456436,-0.148848,-0.481059,...,-0.149486,-0.437273,-0.429515,-0.176517,-0.396893,-0.450565,-0.403872,-0.147028,2,0
2,-0.338311,-0.321432,-0.352675,-0.212496,-0.397089,-0.293996,-0.145525,-0.319485,-0.112567,-0.357485,...,-0.113311,-0.297837,-0.288542,-0.141221,-0.251301,-0.312484,-0.258899,-0.110819,3,0
3,-0.510270,-0.497168,-0.507695,-0.251864,-0.542314,-0.461148,-0.187507,-0.482323,-0.155707,-0.504417,...,-0.156323,-0.463631,-0.456162,-0.183189,-0.424413,-0.476666,-0.431276,-0.153873,4,0
4,-0.510270,-0.497168,-0.507695,-0.251032,-0.542314,-0.461148,-0.186638,-0.482323,-0.154814,-0.504417,...,-0.155433,-0.463631,-0.456162,-0.182320,-0.424413,-0.476666,-0.431276,-0.152981,5,0


In [4]:
flabel=data_all[['Recnum','Fraud']]

In [5]:
variables=data_all.drop(['Fraud'],axis=1)

In [6]:
variables_train=variables[(variables['Recnum']<=84299)&(variables['Recnum']>=3338)]

In [7]:
variables_oot=variables[variables['Recnum']>84299]

In [8]:
flabel_train=flabel[(flabel['Recnum']<=84299)&(flabel['Recnum']>=3338)]

In [9]:
flabel_oot=flabel[flabel['Recnum']>84299]

In [10]:
variables_train=variables_train.drop(['Recnum'],axis=1)

In [11]:
variables_oot=variables_oot.drop(['Recnum'],axis=1)

In [12]:
flabel_train=flabel_train.drop(['Recnum'],axis=1)

In [13]:
flabel_oot=flabel_oot.drop(['Recnum'],axis=1)

In [14]:
flabel_train=flabel_train.iloc[:]['Fraud']

In [15]:
flabel_oot=flabel_oot.iloc[:]['Fraud']

In [16]:
x_train,x_test,y_train,y_test=train_test_split(variables_train,flabel_train,test_size=0.3)

In [17]:
def FDR(x,y,model):
    pred = model.predict_proba(x)[:,1]
    y_df = pd.DataFrame(y)
    y_df['pred']=pred
    top=int(len(y_df)*0.03)
    numbads=sum(y_df['Fraud']==1)
    fdr=y_df.sort_values(by='pred',ascending=False).head(top).Fraud.sum()/numbads
    return fdr

In [18]:
kernal = []
C=[]
gamma=[]
train = []
test = []
ott = []

In [19]:
start = time.time()
clf = svm.SVC(kernel='linear',probability=True,C=0.1)
clf.fit(x_train, y_train)
end = time.time()

In [20]:
C.append(0.1)
gamma.append("-")
train.append(FDR(x_train,y_train,clf))
test.append(FDR(x_test,y_test,clf))
ott.append(FDR(variables_oot,flabel_oot,clf))
print("SVM",0.1, end-start)

SVM 0.1 30.404056549072266


In [21]:
# linear 
Cs = [1, 10]
for c in Cs:
    start = time.time()
    clf = svm.SVC(kernel='linear',probability=True,C=c)
    clf.fit(x_train, y_train)
    end = time.time()
    kernal.append("linear")
    C.append(c)
    gamma.append("-")
    train.append(FDR(x_train,y_train,clf))
    test.append(FDR(x_test,y_test,clf))
    ott.append(FDR(variables_oot,flabel_oot,clf))
    print("SVM: linear",c,"Time:", end - start)

SVM: linear 1 Time: 38.506187200546265
SVM: linear 10 Time: 182.66185855865479


In [22]:
kernal.append("linear")

In [23]:
# rbf variable combination
Cs = [0.01,1,10]
gammas = ["auto","scale"]
rbf_C_gammas = []
for i in Cs:
    for j in gammas:
        rbf_C_gammas.append((i,j))

In [24]:
# rbf 
for p in rbf_C_gammas:
    start = time.time()
    clf = svm.SVC(kernel='rbf', probability=True,C=p[0],gamma=p[1])
    clf.fit(x_train, y_train)
    end = time.time()
    kernal.append("rbf")
    C.append(p[0])
    gamma.append(p[1])
    train.append(FDR(x_train,y_train,clf))
    test.append(FDR(x_test,y_test,clf))
    ott.append(FDR(variables_oot,flabel_oot,clf))
    print("SVM",c, end - start)

SVM 10 39.9853732585907
SVM 10 39.440290689468384
SVM 10 46.30721688270569
SVM 10 50.87651968002319
SVM 10 63.144248723983765
SVM 10 61.94950556755066


In [25]:
# poly variable combination
Cs = [0.01, 1]
gammas = ["auto","scale"]
rbf_C_gammas = []
for i in Cs:
    for j in gammas:
        rbf_C_gammas.append((i,j))

In [26]:
# poly
for p in rbf_C_gammas:
    start = time.time()
    clf = svm.SVC(kernel='poly', probability=True,C=p[0],gamma=p[1])
    clf.fit(x_train, y_train)
    end = time.time()
    kernal.append("poly")
    C.append(p[0])
    gamma.append(p[1])
    train.append(FDR(x_train,y_train,clf))
    test.append(FDR(x_test,y_test,clf))
    ott.append(FDR(variables_oot,flabel_oot,clf))
    print("SVM",c, end - start)

SVM 10 29.528841733932495
SVM 10 39.942195415496826
SVM 10 51.892720222473145
SVM 10 191.4522421360016


In [27]:
# sigmoid
# for p in rbf_C_gammas:
    #start = time.time()
    #clf = svm.SVC(kernel='sigmoid', probability=True,C=p[0],gamma=p[1])
    #clf.fit(x_train, y_train)
    #end = time.time()
    #kernal.append("sigmoid")
    #C.append(p[0])
    #gamma.append(p[1])
    #train.append(FDR(x_train,y_train,clf))
    #test.append(FDR(x_test,y_test,clf))
    #ott.append(FDR(variables_oot,flabel_oot,clf))
    #print("SVM",c, end - start)

In [28]:
result = pd.DataFrame()
result["kernal"] = ["linear","linear","linear","rbf","rbf","rbf","rbf","rbf","rbf","poly","poly","poly","poly"]
result["C"] = C
result["gamma"] = gamma
result["train"] = train
result["test"] = test
result["ott"] = ott

In [29]:
result

,kernal,C,gamma,train,test,ott
0,linear,0.10,-,0.670588,0.684982,0.357542
1,linear,1.00,-,0.668908,0.695971,0.351955
2,linear,10.00,-,0.667227,0.695971,0.346369
3,rbf,0.01,auto,0.640336,0.619048,0.525140
4,rbf,0.01,scale,0.658824,0.604396,0.480447
5,rbf,1.00,auto,0.675630,0.637363,0.480447
6,rbf,1.00,scale,0.736134,0.703297,0.525140
7,rbf,10.00,auto,0.710924,0.644689,0.463687
8,rbf,10.00,scale,0.784874,0.750916,0.536313
9,poly,0.01,auto,0.658824,0.663004,0.312849


In [30]:
result.to_csv("svm_result.csv")